In [0]:
import yaml

with open('params.yml', 'r') as file:
    params = yaml.safe_load(file)


data_params = params.get('data_params')
CATALOG = data_params.get('catalog')
SCHEMA = data_params.get('schema')

In [0]:
from datetime import datetime
import random
from pyspark.sql import Row
from faker import Faker
fake = Faker()
route_data = []
sf_sampled = spark.table(f"{CATALOG}.{SCHEMA}.sf_mobilelocations")
field_technicians = spark.table(f"{CATALOG}.{SCHEMA}.field_technicians")

lons = sf_sampled.select('lon').toPandas()['lon'].tolist()
lats = sf_sampled.select('lat').toPandas()['lat'].tolist()
for tech in field_technicians.collect():
    if random.randint(1, 7) <= 2:  # 2/7 chance of being off work
        route_data.append(Row(
            guid=tech.tech_id,
            name=tech.name,
            lat=None,
            lon=None,
            stop_number=None,
            random_text="not scheduled",
            date=datetime.today().strftime('%Y-%m-%d'),
        ))
    else:
        num_stops = random.randint(1, 15)
        base_lat = random.choice(lats)
        base_lon = lons[lats.index(base_lat)]
        for stop_num in range(1, num_stops + 1):
            route_data.append(Row(
                guid=tech.tech_id,
                name=tech.name,
                lat=base_lat + random.uniform(-0.01, 0.01),
                lon=base_lon + random.uniform(-0.01, 0.01),
                stop_number=stop_num,
                random_text=fake.text(max_nb_chars=50),
                date=datetime.today().strftime('%Y-%m-%d'),
            ))

routes_df = spark.createDataFrame(route_data)
routes_df.write.mode("append").saveAsTable(f"{CATALOG}.{SCHEMA}.fieldtech_route")

display(routes_df)